# Boite à outils pour sbi
___

Pour le sommaire, voir le Outline sur VSCode


## Fonctions pour modifier les paramètres de PhysiCell
___

### Fonction pour modifier PhysiCell_settings.xml
___

In [1]:
import xml.etree.ElementTree as ET

def modify_xml_value(file_path, tag_path, new_value):
    tree = ET.parse(file_path)
    root = tree.getroot()
    element = root
    for tag in tag_path:
        element = element.find(tag)
        if element is None:
            print(f"Balise '{tag}' non trouvée dans le chemin spécifié.")
            return False
    element.text = str(new_value)
    tree.write(file_path, encoding="utf-8", xml_declaration=True)
    return True

#### Exemples d'utilisation

In [ ]:
# Exemple 1 : modifier le temps total de simulation
tag_path = ["overall", "max_time"]
new_value = 1440
modify_xml_value("config/PhysiCell_settings.xml", tag_path, new_value)

# Exemple 2 : modifier le pas de temps de enregistrement des données
tag_path = ["save", "SVG", "interval"]
new_value = 200
modify_xml_value("config/PhysiCell_settings.xml", tag_path, new_value)
tag_path = ["save", "full_data", "interval"]
modify_xml_value("config/PhysiCell_settings.xml", tag_path, new_value)

# Exemple 3 : modifier la force de répulsion entre les cellules de type "follower cell"
tag_path = ["cell_definitions", "cell_definition[@name='follower cell']", "phenotype","mechanics", "cell_cell_repulsion_strength"]
new_value = 90
modify_xml_value("config/PhysiCell_settings.xml", tag_path, new_value)
# Utiliser [@attribut='valeur'] pour les balises avec attributs comment cell_definition


### Fonction pour modifier rules.csv
___

In [ ]:
import csv
def change_csv_value(csv_file_path, row_index, column_index, new_value):
    # Read the CSV file into a list of lists
    with open(csv_file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        data = list(reader)
    # Update the value of the specified cell
    if 0 <= row_index < len(data) and 0 <= column_index < len(data[row_index]):
        data[row_index][column_index] = str(new_value)

        # Save the modified data to the CSV file
        with open(csv_file_path, 'w', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerows(data)

    else:
        print(f"Invalid row index ({row_index}) or column index ({column_index})")


#### Exemples d'utilisation

In [ ]:
# Exemple de fichier rules.csv

#cancer,pressure,decreases,cycle entry,0.0,1,4,0
#cancer,oxygen,increases,cycle entry,0.0007,21.5,4,0
#cancer,oxygen,decreases,necrosis,0,3.75,8,0

# Exemple 1 : modifier la saturation value de la première règle à 1 (au lieu de 0)
change_csv_value('./config/rules.csv', 0, 4, 1) # (0,4) étant la ligne 0 et la colonne 4

# Exemple 2 : modifier le Half-max de la deuxième règle à 20 (au lieu de 21.5)
change_csv_value('./config/rules.csv', 1, 5, 20) # (1,5) étant la ligne 1 et la colonne 5

# Exemple 3 : modifier la puissance de Hill de la deuxième règle à 5 (au lieu de 4)
change_csv_value('./config/rules.csv', 1, 6, 5) 

# Exemple 4 : modifier le apply to dead de la troisème règle à 1(True) (au lieu de 0(False))
change_csv_value('./config/rules.csv', 2, 7, 1) 

## Fonctions pour récuperer des données numériques résultat
___

### Fonction qui récupère le nombre total de cellules
___
Elle va chercher cette information dans un fichier .svg

In [2]:
import xml.etree.ElementTree as ET

def get_agent_count(svg_file_path):
    # Parse the SVG file
    tree = ET.parse(svg_file_path)
    root = tree.getroot()
    
    # Define the namespace
    namespaces = {'svg': 'http://www.w3.org/2000/svg'}
    
    # Find all text elements
    text_elements = root.findall('.//svg:text', namespaces)
    
    # Iterate through text elements to find the one containing the agent count
    for elem in text_elements:
        if 'agents' in elem.text:
            # Extract the number of agents from the text
            agent_count = int(''.join(filter(str.isdigit, elem.text)))
            return agent_count
    
    # If no agent count was found, return None or raise an exception
    return None

#### Exemple d'utilisation

In [5]:
# Exemple 1 : obtenir le nombre d'agents dans le fichier final.svg
agent_count1 = get_agent_count('/home/aymeric/tumor-simulation/PhysiCell/output/final.svg')

# Exemple 2 : obtenir le nombre d'agents dans le fichier initial.svg
agent_count2 = get_agent_count('/home/aymeric/tumor-simulation/PhysiCell/output/initial.svg')

print(f"Nombre d'agents dans le fichier final.svg : {agent_count1}")
print(f"Nombre d'agents dans le fichier initial.svg : {agent_count2}")

Nombre d'agents dans le fichier final.svg : 577
Nombre d'agents dans le fichier initial.svg : 513


### Fonction qui récupère le nombre total de cellules d'une même population selon l'id
___
Elle va chercher cette information dans un fichier .mat 
L'id de chaque population est indiqué dans Cell Types

In [6]:
import numpy as np
import scipy.io

def get_count_by_id(path_mat, id_number):
    mat = scipy.io.loadmat(path_mat)
    id = mat['cells'][5]
    
    mask_id1 = (id == id_number)

    count_id1 = np.count_nonzero(mask_id1)

    return count_id1

#### Exemple d'utilisation

In [9]:
# Exemple 1 : obtenir le nombre de cellules de type 0 dans le fichier output00000001_cells.mat
count_id1 = get_count_by_id('/home/aymeric/tumor-simulation/PhysiCell/output/output00000001_cells.mat', 0)

# Exemple 2 : obtenir le nombre de cellules de type 2 dans le fichier final_cells.mat
count_id2 = get_count_by_id('/home/aymeric/tumor-simulation/PhysiCell/output/final_cells.mat', 2)

print(f"Nombre de cellules de type 1 dans le fichier output00000001_cells.mat : {count_id1}")
print(f"Nombre de cellules de type 2 dans le fichier final_cells.mat : {count_id2}")

Nombre de cellules de type 1 dans le fichier output00000001_cells.mat : 528
Nombre de cellules de type 2 dans le fichier final_cells.mat : 0


### Fonction qui retourne la densité radiale

In [13]:
import numpy as np
def calculate_distances(points, center):
    distances = np.sqrt((points[:, 0] - center[0])**2 + (points[:, 1] - center[1])**2)
    return distances

def radial_density(distances, step=10, max_distance=500):
    # Créer les bins avec une taille allant jusqu'à 500 et un step de 10
    bins = np.arange(0, max_distance + step, step)
    counts, _ = np.histogram(distances, bins)
    areas = np.pi * (bins[1:]**2 - bins[:-1]**2)
    densities = counts / areas
    return bins[:-1], densities

def get_radial_density(path_mat):
    mat = scipy.io.loadmat(path_mat)
    X = mat['cells'][1]
    Y = mat['cells'][2]
    points = np.column_stack((X, Y));
    center = np.array([0,0]);
    distances = calculate_distances(points, center)
    radii, densities = radial_density(distances)
    resultat = [elem for pair in zip(radii, densities) for elem in pair]
    return resultat


#### Exemple d'utilisation

In [14]:
print(get_radial_density('/home/aymeric/tumor-simulation/PhysiCell/output/final_cells.mat'))

[0, 0.00954929658551372, 10, 0.009549296585513721, 20, 0.005729577951308232, 30, 0.008185111359011761, 40, 0.007427230677621782, 50, 0.007234315595086152, 60, 0.007590466516690393, 70, 0.007639437268410976, 80, 0.007864126599834828, 90, 0.007371386837940416, 100, 0.0071240784050657915, 110, 0.007750153750561861, 120, 0.00751211331393746, 130, 0.0076630157784986636, 140, 0.0075735800505798475, 150, 0.0036965019040698273, 160, 0.0006752027888747075, 170, 0.0, 180, 0.0, 190, 0.0, 200, 0.0, 210, 0.0, 220, 0.0, 230, 0.0, 240, 0.0, 250, 0.0, 260, 0.0, 270, 0.0, 280, 0.0, 290, 0.0, 300, 0.0, 310, 0.0, 320, 0.0, 330, 0.0, 340, 0.0, 350, 0.0, 360, 0.0, 370, 0.0, 380, 0.0, 390, 0.0, 400, 0.0, 410, 0.0, 420, 0.0, 430, 0.0, 440, 0.0, 450, 0.0, 460, 0.0, 470, 0.0, 480, 0.0, 490, 0.0]


### Fonction qui récupère la position X et Y d'une cellule
___
Cette fonction est utile si on veut étudier le déplacement d'une seule cellule

In [ ]:
import scipy.io

def get_pos(path_mat):
    mat = scipy.io.loadmat(path_mat)
    X = mat['cells'][1]  
    Y = mat['cells'][2]  

    return X[0], Y[0]

## Fonction de simulation
___
Cette fonction a pour objectif de créer des données en effectuant des simulations tout en faisant varier les paramètres en entrée.

Voici un exemple :

### Exemple de fonction de simulation
___
Cette fonction va modifier la vitesse, la force d'adhésion, de répulsion et distance equilibre des cellules "cancer" et va retourner comme résultat la densité radiale.

In [28]:
import torch
import subprocess

def simulation(params):

    # Vérifier si les paramètres sont un vecteur ou une matrice
    # Si c'est un vecteur, n=1, simulation(torch.tensor([0.5, 0.5, 0.05, 10, 1.25]))
    # Si c'est une matrice, n=nombre de lignes, simulation(torch.tensor([[0.5, 0.5, 0.05, 10, 1.25], [0.6, 0.6, 0.06, 11, 1.26]]))
    if len(params.shape) == 1:
        n = 1
        num_params = len(params)
    else:
        n, num_params = params.shape
    
    results = []
    
    for i in range(n):
        if n == 1:
            current_params = params
        else:
            current_params = params[i]

        # Liste des chemins XML et des noms de paramètres
        xml_paths = [
            ["cell_definitions", "cell_definition[@name='cancer']", "phenotype", "motility", "speed"],
            ["cell_definitions", "cell_definition[@name='cancer']", "phenotype", "mechanics", "cell_cell_adhesion_strength"],
            ["cell_definitions", "cell_definition[@name='cancer']", "phenotype", "mechanics", "cell_cell_repulsion_strength"],
            ["cell_definitions", "cell_definition[@name='cancer']", "phenotype", "mechanics", "relative_maximum_adhesion_distance"]
        ]

        # Modification des valeurs XML pour chaque paramètre disponible
        for j in range(min(num_params, len(xml_paths))):
            modify_xml_value("config/PhysiCell_settings.xml", xml_paths[j], current_params[j].item())

        subprocess.run(["./project"], stdout=subprocess.DEVNULL)

        mat_filename = '/home/aymeric/tumor-simulation/PhysiCell/output/final_cells.mat'
        pos = get_radial_density(mat_filename)
        results.append(torch.tensor(pos))

    results = torch.cat(results)
    return results.flatten()

#### Exemple d'utilisation

In [27]:
# Exemple 1 : exécuter la simulation avec un vecteur de paramètres
print(simulation(torch.tensor([0.5, 0.05, 10, 1.25])))

# Exemple 2 : exécuter la simulation avec une matrice de paramètres
print(simulation(torch.tensor([[0.5, 0.05, 10, 1.25],[0.5, 0.05, 10, 1.25]])))


tensor([0.0000e+00, 9.5493e-03, 1.0000e+01, 8.4883e-03, 2.0000e+01, 6.3662e-03,
        3.0000e+01, 8.1851e-03, 4.0000e+01, 7.0736e-03, 5.0000e+01, 8.3918e-03,
        6.0000e+01, 6.8559e-03, 7.0000e+01, 7.4272e-03, 8.0000e+01, 7.4896e-03,
        9.0000e+01, 7.7064e-03, 1.0000e+02, 7.4272e-03, 1.1000e+02, 7.8885e-03,
        1.2000e+02, 7.6394e-03, 1.3000e+02, 6.8378e-03, 1.4000e+02, 7.3541e-03,
        1.5000e+02, 6.0582e-03, 1.6000e+02, 6.7520e-04, 1.7000e+02, 9.0946e-05,
        1.8000e+02, 0.0000e+00, 1.9000e+02, 0.0000e+00, 2.0000e+02, 0.0000e+00,
        2.1000e+02, 0.0000e+00, 2.2000e+02, 0.0000e+00, 2.3000e+02, 0.0000e+00,
        2.4000e+02, 0.0000e+00, 2.5000e+02, 0.0000e+00, 2.6000e+02, 0.0000e+00,
        2.7000e+02, 0.0000e+00, 2.8000e+02, 0.0000e+00, 2.9000e+02, 0.0000e+00,
        3.0000e+02, 0.0000e+00, 3.1000e+02, 0.0000e+00, 3.2000e+02, 0.0000e+00,
        3.3000e+02, 0.0000e+00, 3.4000e+02, 0.0000e+00, 3.5000e+02, 0.0000e+00,
        3.6000e+02, 0.0000e+00, 3.7000e+

### Template de fonction simulation
___

In [ ]:
import torch
import subprocess

def simulation(params):

    # Vérifier si les paramètres sont un vecteur ou une matrice
    # Si c'est un vecteur, n=1, simulation(torch.tensor([0.5, 0.05, 10, 1.25]))
    # Si c'est une matrice, n=nombre de lignes, simulation(torch.tensor([[0.5, 0.05, 10, 1.25], [0.6, 0.06, 11, 1.26]])) Attention, une parenthèse en plus
    if len(params.shape) == 1:
        n = 1
        num_params = len(params)
    else:
        n, num_params = params.shape
    
    results = []
    
    for i in range(n):
        if n == 1:
            current_params = params
        else:
            current_params = params[i]

        #### MODIFICATION DES PARAMÈTRES DANS LES FICHIERS XML OU CSV

        ##### À COMPLÉTER #####

        #####

        #### LANCER LA SIMULATION 
        subprocess.run(["./project"], stdout=subprocess.DEVNULL)
        ####

        #### EXTRACTION DES DONNEES RESULTANTES

        ##### À COMPLÉTER #####

        #####


    results = torch.cat(results)
    return results.flatten()

## Inférence
___

Pour faire de l'inférence avec PhysiCell, il faut avoir chargé un projet et l'avoir compilé avant de pouvoir lancer les étapes suivantes. Les paramètres constant (ceux qu'on ne cherche pas à faire varier) n'ont pas forcément les même valeur d'un projet PhysiCell à un autre.

### Initialisation de l'a priori
___

In [ ]:
import torch
from sbi.inference import prepare_for_sbi

from sbi import utils as utils
from sbi import analysis as analysis

# Définir les bornes inférieures pour chaque dimension
low = torch.tensor([0, 0, 0, 0]) # 4 paramètres donc 4 bornes min

# Définir les bornes supérieures pour chaque dimension
high = torch.tensor([1, 1, 10, 2]) # 4 paramètres donc 4 bornes max

# Créer le a priori uniforme avec les bornes spécifiées
prior = utils.BoxUniform(low=low, high=high)

simulator, prior = prepare_for_sbi(simulation, prior)

### Choix du type d'inférence
___

Plusieurs méthodes disponibles dans la librairie sbi.
Perspective à approfondir

In [ ]:
from sbi.inference import SNPE

inference = SNPE(prior=prior)

### Génération de données fictives du simulateur
___

Choisir le nombre de simulation, plus il y en a, plus le modèle pourra bien s'entrainer (Attention, demande beaucoup de temps)

In [ ]:
from sbi.inference import simulate_for_sbi

theta, x = simulate_for_sbi(simulator, proposal=prior, num_simulations=5) # 5 simulations pour l'exemple

### Entrainement du modèle
___

Assez rapide, n'a jamais duré plus de 10 minutes

In [ ]:
# On ajoute les simulations à l'objet inference
inference = inference.append_simulations(theta, x)

# On entraîne le réseau de neurones
density_estimator = inference.train()

### Enregistrer les données simulées
___
Etape pour sauvegarder dans des fichiers les données simulées qui ont demandé beaucoup de calculs

In [ ]:
import pickle
# Attention à bien changer les chemins pour sauvegarder les fichiers.pkl

with open("/home/aymeric/tumor-simulation/NN/leader/simulated_data1000.pkl", "wb") as handle:
    pickle.dump((theta, x), handle)
with open("/home/aymeric/tumor-simulation/NN/leader/density_estimator1000.pkl", "wb") as handle:
    pickle.dump(density_estimator, handle)

### Charger des données depuis des fichiers .pkl
___

In [ ]:
import pickle
# Attention à bien changer les chemins pour charger les bons fichiers.pkl

with open("/home/aymeric/tumor-simulation/NN/leader/simulated_data1000.pkl", "rb") as handle:
    theta, x = pickle.load(handle)
with open("/home/aymeric/tumor-simulation/NN/leader/density_estimator1000.pkl", "rb") as handle:
    density_estimator = pickle.load(handle)

### Construction du posterieur
___

In [ ]:
posterior = inference.build_posterior(density_estimator)

In [ ]:
theta = torch.tensor([0.5, 0.5, 10, 1.25]) # Paramètre de départ que l'on cherche à prédire
x_o = simulation(theta) # Résultat de la simulation pour le paramètre de départ

In [ ]:
# On génère 10000 échantillons à partir du posterior (très rapide car on utilise un NN)
posterior_samples = posterior.sample((10000,), x=x_o) 

# plot posterior samples
_ = analysis.pairplot(
    posterior_samples, limits=[[0, 1], [0, 1], [0, 10], [0, 2]], figsize=(5, 5) # bornes min et max pour chaque paramètre à changer
)

## Vérification des résultats
___

### Fonctions de mesures d'erreurs
___


In [ ]:
import numpy as np

def calculate_mse(list1, list2):
    """Calculer l'erreur quadratique moyenne entre deux listes."""
    return np.mean((np.array(list1) - np.array(list2)) ** 2)

def calculate_nmse(list1, list2):
    """Calculer l'erreur quadratique moyenne normalisée."""
    mse = calculate_mse(list1, list2)
    variance = np.var(list1)
    return mse / variance if variance != 0 else float('inf')

def calculate_relative_mse(list1, list2):
    """Calculer l'erreur quadratique relative."""
    mse = calculate_mse(list1, list2)
    mean1 = np.mean(list1)
    mean2 = np.mean(list2)
    relative_mse = mse / ((mean1 + mean2) / 2)
    return relative_mse

def calculate_mae(list1, list2):
    """Calculer l'erreur absolue moyenne entre deux listes."""
    return np.mean(np.abs(np.array(list1) - np.array(list2)))

def calculate_log_mse(list1, list2):
    """Calculer le logarithme de l'erreur quadratique moyenne."""
    mse = calculate_mse(list1, list2)
    return np.log(mse + 1e-10)  # Éviter log(0) avec une petite constante

In [ ]:
print(posterior_samples.median(dim=0).values)
predicted_tensor = simulation(posterior_samples.median(dim=0).values)
print(predicted_tensor)


# Exemples de listes
list1 = x_o.numpy()
list2 = predicted_tensor.numpy()

# Calcul des différentes métriques
mse = calculate_mse(list1, list2)
nmse = calculate_nmse(list1, list2)
relative_mse = calculate_relative_mse(list1, list2)
mae = calculate_mae(list1, list2)
log_mse = calculate_log_mse(list1, list2)

# Affichage des résultats
print(f"Erreur quadratique moyenne (MSE) : {mse}")
print(f"Erreur quadratique moyenne normalisée (NMSE) : {nmse}")
print(f"Erreur quadratique relative : {relative_mse}")
print(f"Erreur absolue moyenne (MAE) : {mae}")
print(f"Logarithme de l'erreur quadratique moyenne (Log MSE) : {log_mse}")